# Preparing raw dataset

In [1]:
import pandas as pd

In [39]:
df = pd.read_parquet("hf://datasets/Amirkid/MedQuad-dataset/data/train-00000-of-00001-744a44ed05ce0a48.parquet")
df

In [41]:
def process_medical_data(raw_data):
    questions = []
    answers = []
    for i in range(0, len(raw_data), 2):  # Step by 2 to get question and answer pairs
        question = raw_data.iloc[i]['text'].strip()  # Get the question
        answer = df.iloc[i + 1]['text'].strip() if (i + 1) < len(raw_data) else ""  # Get the answer
        
        questions.append(question)
        answers.append(answer)
    
    # Create a new DataFrame
    new_df = pd.DataFrame({
        'question': questions,
        'answer': answers
    })

    return new_df

new_df = process_medical_data(df)

In [42]:
new_df
# new_df.to_csv("data/raw/train.csv", index=False)

,question,answer
0,What is (are) Neck Injuries and Disorders ?,"Any part of your neck muscles, bones, joints,..."
1,What is (are) Heel Injuries and Disorders ?,Heel problems are common and can be painful. O...
2,Do you have information about CT Scans,Summary : Computed tomography (CT) is a type o...
3,What is (are) Self-harm ?,Selfharm refers to a person's harming their ow...
4,Do you have information about Seniors' Health,Summary : People in the U.S. are living longer...
...,...,...
16395,What is (are) Parasites - Fascioliasis (Fascio...,Fascioliasis is an infectious disease caused b...
16396,Who is at risk for Parasites - Fascioliasis (F...,Fascioliasis occurs in many areas of the world...
16397,How to diagnose Parasites - Fascioliasis (Fasc...,The standard way to be sure a person is infect...
16398,What are the treatments for Parasites - Fascio...,The first step is to make sure the diagnosis i...


In [48]:
df_qa = pd.read_csv('data/raw/train.csv')
df_symptoms = pd.read_csv('data/raw/Symptom2Disease.csv')
combined_df = pd.DataFrame({
    'label': pd.concat([df_qa['question'], df_symptoms['label']]).reset_index(drop=True),
    'text': pd.concat([df_qa['answer'], df_symptoms['text']]).reset_index(drop=True)
})

combined_df.to_csv('data/raw/combined_dataset.csv', index=False)

In [49]:
combined_df

,label,text
0,What is (are) Neck Injuries and Disorders ?,"Any part of your neck muscles, bones, joints,..."
1,What is (are) Heel Injuries and Disorders ?,Heel problems are common and can be painful. O...
2,Do you have information about CT Scans,Summary : Computed tomography (CT) is a type o...
3,What is (are) Self-harm ?,Selfharm refers to a person's harming their ow...
4,Do you have information about Seniors' Health,Summary : People in the U.S. are living longer...
...,...,...
17595,diabetes,I'm shaking and trembling all over. I've lost ...
17596,diabetes,"Particularly in the crevices of my skin, I hav..."
17597,diabetes,I regularly experience these intense urges and...
17598,diabetes,"I have trouble breathing, especially outside. ..."


# Preprocessing dataset

In [1]:
from src.data_preprocessing import preprocess_and_save
preprocess_and_save()

Loading raw data...
Cleaning training data...
Saving processed data to CSV files...
Data preprocessing completed successfully.


In [3]:
import pandas as pd
preprocessed_df = pd.read_csv('data/processed/train.csv')
preprocessed_df

,label,text
0,neck injury disorder,part neck muscle bone joint tendon ligament ne...
1,heel injury disorder,heel problem common painful often result much ...
2,information ct scan,summary computed tomography ct type imaging us...
3,selfharm,selfharm refers person harming body purpose 1 ...
4,information senior health,summary people u living longer ever many senio...
...,...,...
17595,diabetes,im shaking trembling ive lost sense taste smel...
17596,diabetes,particularly crevice skin skin rash irritation...
17597,diabetes,regularly experience intense urge want urinate...
17598,diabetes,trouble breathing especially outside start fee...


In [4]:
from datasets import Dataset
train = Dataset.from_pandas(preprocessed_df)

# Load pre-trained model Bert base and tokenizer 

In [5]:
train

Dataset({
    features: ['label', 'text'],
    num_rows: 17600
})

In [6]:
train[0]

{'label': 'neck injury disorder',
 'text': 'part neck muscle bone joint tendon ligament nerve cause neck problem neck pain common pain may also come shoulder jaw head upper arm muscle strain tension often cause neck pain problem usually overuse sitting computer long sometimes strain neck muscle sleeping awkward position overdoing exercise fall accident including car accident another common cause neck pain whiplash soft tissue injury neck also called neck sprain strain treatment depends cause may include applying ice taking pain reliever getting physical therapy wearing cervical collar rarely need surgery'}

In [2]:
# check torch using gpu or not
import torch
torch.cuda.is_available()
print(torch.cuda.get_device_name(0))

NVIDIA GeForce RTX 4050 Laptop GPU


In [20]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, TrainerCallback
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

# Load the dataset
df = pd.read_csv('data/raw/train.csv')

# Preprocessing
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Define custom dataset
class InjuryDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            truncation=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-large-cased')
model = BertForSequenceClassification.from_pretrained('bert-large-cased', num_labels=len(df['label'].unique()))

BATCH_SIZE = 8
MAX_LEN = 128
GRADIENT_ACCUMULATION_STEPS = 8
EPOCHS = 10
LEARNING_RATE = 2e-5

# Split the dataset
train_texts, val_texts, train_labels, val_labels = train_test_split(df['text'], df['label'], test_size=0.1, random_state=42)

# Create DataLoader
train_dataset = InjuryDataset(train_texts.tolist(), train_labels.tolist(), tokenizer, MAX_LEN)
val_dataset = InjuryDataset(val_texts.tolist(), val_labels.tolist(), tokenizer, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

# Define metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    return {"accuracy": accuracy}

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    max_grad_norm=0.5,  # Gradient clipping
    warmup_steps=1000,  # Increased warmup steps
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_steps=100,
    evaluation_strategy="steps",
    save_strategy='steps',
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    seed=42,
    lr_scheduler_type="linear",
    optim="adamw_torch",
)

# Define a callback to print training logs
class PrinterCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if state.is_local_process_zero and logs is not None:
            if 'loss' in logs:
                print(f"Step {state.global_step}: Training Loss: {logs['loss']:.5f}")
            else:
                print(f"Step {state.global_step}: Logs: {logs}")

# Enable gradient checkpointing to save memory
model.gradient_checkpointing_enable()

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[PrinterCallback()]
)

# Clear CUDA cache before training
torch.cuda.empty_cache()

# Train the model
trainer.train()

# Save the model and tokenizer
trainer.save_model('models/bert-medical-qa')
tokenizer.save_pretrained('models/bert-medical-qa')

# Evaluate the model and show metrics
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\shiva\miniconda3\envs\gpu-env\lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
c:\Users\shiva\miniconda3\envs\gpu-env\lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
  0%|          | 0/2300 [00:00<?, ?it/s]c:\Users\shiva\miniconda3\envs\gpu-env\lib\site-packages\torch\utils\checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Pleas

Step 10: Training Loss: 9.76430
{'loss': 9.7643, 'grad_norm': inf, 'learning_rate': 4.5e-07, 'epoch': 0.04}


  1%|          | 20/2300 [10:38<21:24:26, 33.80s/it]

Step 20: Training Loss: 9.73920
{'loss': 9.7392, 'grad_norm': 8.453700065612793, 'learning_rate': 9.5e-07, 'epoch': 0.09}


  1%|▏         | 30/2300 [16:42<23:07:26, 36.67s/it]

...

Step 2300: Logs: {'eval_loss': 10.64493179321289, 'eval_accuracy': 0.011585365853658536, 'eval_runtime': 235.8329, 'eval_samples_per_second': 6.954, 'eval_steps_per_second': 1.739, 'epoch': 9.97289972899729}
{'eval_loss': 10.64493179321289, 'eval_accuracy': 0.011585365853658536, 'eval_runtime': 235.8329, 'eval_samples_per_second': 6.954, 'eval_steps_per_second': 1.739, 'epoch': 9.97}


100%|██████████| 2300/2300 [25:35:51<00:00, 40.07s/it]

Step 2300: Logs: {'train_runtime': 92151.63, 'train_samples_per_second': 1.602, 'train_steps_per_second': 0.025, 'total_flos': 3.60298513711104e+16, 'train_loss': 8.716842704441236, 'epoch': 9.97289972899729}
{'train_runtime': 92151.63, 'train_samples_per_second': 1.602, 'train_steps_per_second': 0.025, 'train_loss': 8.716842704441236, 'epoch': 9.97}



100%|██████████| 410/410 [03:55<00:00,  1.74it/s]

Step 2300: Logs: {'eval_loss': 10.700302124023438, 'eval_accuracy': 0.011585365853658536, 'eval_runtime': 235.9645, 'eval_samples_per_second': 6.95, 'eval_steps_per_second': 1.738, 'epoch': 9.97289972899729}
Evaluation results: {'eval_loss': 10.700302124023438, 'eval_accuracy': 0.011585365853658536, 'eval_runtime': 235.9645, 'eval_samples_per_second': 6.95, 'eval_steps_per_second': 1.738, 'epoch': 9.97289972899729}
